In [24]:
import pandas as pd

import numpy as np 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding, Dense, LSTM, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import  DistilBertTokenizer
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments

In [3]:
encoder = LabelEncoder()

Loading Dataset

In [4]:
train = pd.read_csv("D:\Datasets\movie-genre-prediction\\train.csv")

Deleting the unnecessary Columns

In [5]:
dataset = train.drop(columns=['Unnamed: 0','movie_name','id'],axis=1)

Data Preparation

Encoding the labels

In [6]:
dataset['genre_enc'] = encoder.fit_transform(dataset['genre'])

In [7]:
dataset.head()

,synopsis,genre,genre_enc
0,A young scriptwriter starts bringing valuable ...,fantasy,4
1,A director and her friends renting a haunted h...,horror,5
2,This is an educational video for families and ...,family,3
3,Scientists working in the Austrian Alps discov...,scifi,8
4,Buy Day - Four Men Widely - Apart in Life - By...,action,0


In [8]:
tokenizer = Tokenizer()

In [9]:
tokenizer.fit_on_texts(dataset['synopsis'])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequenced_text = tokenizer.texts_to_sequences(dataset['synopsis'])

In [10]:
print(max([len(txt) for txt in sequenced_text]))

70


Applying Padding

In [11]:
pad_sequence = pad_sequences(sequenced_text, maxlen=70)

Splitting the dataset

In [16]:
x_train,x_valid, y_train, y_valid = train_test_split(pad_sequence,dataset['genre_enc'])

Data Preparation for Transformers

Converting data into tensorflow datasets

In [18]:
tf_train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
tf_valid = tf.data.Dataset.from_tensor_slices((x_valid,y_valid))

Creating the BERT Model

In [20]:
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: Exception encountered when calling layer "tf_bert_for_sequence_classification" (type TFBertForSequenceClassification).

in user code:

    File "c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_tf_utils.py", line 1557, in run_call_with_unpacked_inputs  *
        return func(self, **unpacked_inputs)
    File "c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_tf_bert.py", line 1569, in call  *
        outputs = self.bert(
    File "c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\adity\AppData\Local\Temp\__autograph_generated_filemznqmxu4.py", line 40, in tf__run_call_with_unpacked_inputs
        raise
    File "C:\Users\adity\AppData\Local\Temp\__autograph_generated_file7xm78srg.py", line 127, in tf__call
        embe

In [22]:
bert_model.compile(optimizer='adam',loss='spare_categoricalcrossentropy',metrics=['accuracy'])
bert_model.fit(tf_train)

ValueError: in user code:

    File "c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_tf_utils.py", line 1658, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\adity\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\U